# Word2Vec then RNN

## Load data

In [ ]:
import pandas as pd
from os import path

importDirectory = "./data/preprocessed-train-test/"

train, test, data, contestTest = map(
    lambda filename: pd.read_csv(path.join(importDirectory, filename)), 
    ["train.csv", "test.csv", "all.csv", "contest-test.csv"])

In [ ]:
print("train: {}, test: {}, all: {}, contestTest: {}".format(
    train.shape, test.shape, data.shape, contestTest.shape))

In [ ]:
def splitSentences(dataset):
    return (dataset.comment_text
    .str.replace("[^A-Za-z\s]", "")
    .str.lower()
    .str.split())

In [ ]:
%%time
splitTrain = splitSentences(train)
splitTest = splitSentences(test)

## Sentense lengths

In [ ]:
sentenceLengths = splitTrain.apply(len)
sentenceLengths.describe()

In [ ]:
import matplotlib.pyplot as plt

sentenceLengths.plot.box()
plt.show()

In [ ]:
sentenceLengths.hist(bins=100)
plt.show()

## Load Word2Vec dictionary

In [ ]:
%%time
import gensim

model = gensim.models.KeyedVectors.load_word2vec_format("./external-models/glove.6B/w2v.glove.6B.50.txt")  

In [ ]:
%%time
vocabulary = set(model.wv.vocab.keys())

## Transform sentences to sequences of vectors

In [ ]:
splitTrain.shape

In [ ]:
model.wv["hello", "there"].shape

In [ ]:
%%time
import numpy as np

def wordsToVector(words):
    allowedWords = [word for word in words if word in vocabulary]
    leftWords = allowedWords if allowedWords else ["hello"]
    
    return model.wv[leftWords]

In [ ]:
%%time
w2vTrainFeatures = splitTrain.apply(wordsToVector)

In [ ]:
%%time
w2vTestFeatures = splitTest.apply(wordsToVector)

## Helper to get batches

In [ ]:
np.random.seed(4324) # always remember to reset the seed before using getTrainBatch

def getTrainBatch(size):
    return w2vTrainFeatures.sample(size)